#                                        Reinforcement Learning

### ABSTRACT  📝

Taxi-v3 is the environment that I have chosen to test the Reinforcement Learning models. Here, the agent is the taxi and it has to ensure to
1. Drop off the passanger to the destination
2. Take the shortest distance and save time
3. Ensure passenger safety and follow traffic rules

In Taxi-v3 environment, the agent starts at a random location and 
One **episode** is defined as one sequence of states, actions, and rewards that end with a terminal state


Passenger locations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
    - 4: in taxi
    
    Destinations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
        
    Actions:
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east 
    - 3: move west 
    - 4: pickup passenger
    - 5: dropoff passenger
    
    Rewards: 
    There is a reward of -1 for each action and an additional reward of +20 for delivering the passenger. There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.
    
    Rendering:
    - blue: passenger
    - magenta: destination
    - yellow: empty taxi
    - green: full taxi
    - other letters (R, G, Y and B): locations for passengers and destinations

Gym is a toolkit for developing and comparing RL algorithms. The gym library is a collection of test problems — environments — that can be used to work out reinforcement learning algorithms. These environments have a shared interface, allowing one to write general algorithms.

# Reinforcement Learing | OpenAI | Taxi-V3 🚕

In [1]:
#import the requried libraries
import numpy as np 
import gym            # for Taxi environment
import random         # To generate random numbers

## Create environment using OpenAI 

Here we create Taxi-v3 environment and use it to train our agent

In [2]:
env = gym.make("Taxi-v3")
env.render()             # renders environment

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# Create Qtable 
**Actions**(Columns) and **States**(Rows) of a Qtable can be determined using openAI gym 

In [3]:
action_size = env.action_space.n
print("Action size ", action_size)

Action size  6


In [4]:
state_size = env.observation_space.n
print("State size ", state_size)

State size  500


This means, we've 6 possible actions and 500 possible states

Qtable consists of actions and states. At first, all the values in qtable is initialzed to zero as shown below

# Initialize the Qtable to zeroes  🆒

In [5]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# The Hyperparameters!

## 1. With the baseline parameters

In [9]:
total_episodes = 5000
total_test_episodes = 100
max_steps = 99 
alpha= 0.7 # Learning rate 
gamma = 0.8 # Discounting rate 
epsilon = 1.0 # Exploration rate
min_epsilon = 0.01
max_epsilon = 1.0
decay_rate = 0.01 # Exponential decay rate

If we are in the state of exploration, the action is chosen randomly. In the case of exploitation, we'll take the action with the highest Q-value for that state

## The Q Learning Algorithm

In [10]:
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
        
        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + alpha * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])
                
        # Our new state is state
        state = new_state
        
        # If done : finish episode
        if done == True: 
            break
    
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

In [11]:
env.reset()               # resets the environment and returns a random initial state
rewards = []              

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        # UNCOMMENT IT IF YOU WANT TO SEE OUR AGENT PLAYING
        # env.render()
        #Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

****************************************************
EPISODE  0
Score 6
****************************************************
EPISODE  1
Score 6
****************************************************
EPISODE  2
Score 7
****************************************************
EPISODE  3
Score 9
****************************************************
EPISODE  4
Score 7
****************************************************
EPISODE  5
Score 3
****************************************************
EPISODE  6
Score 6
****************************************************
EPISODE  7
Score 8
****************************************************
EPISODE  8
Score 8
****************************************************
EPISODE  9
Score 10
****************************************************
EPISODE  10
Score 9
****************************************************
EPISODE  11
Score 6
****************************************************
EPISODE  12
Score 7
****************************************************
EPISODE

## Thus, the score over time with the baseline parameter is 7.32

### Our next goal is to improve the result by tuning the hyperparameters  🎯

## -----------------------------------------------------------------------------------------------------------------------------------

# Try tuning hyperparameters to improve the acquried results 🛠

In [16]:
alpha_list = [0.6,0.65,0.7,0.75,0.8,0.9]
gamma_list = [0.5,0.54,0.618,0.71,0.78,0.88]

In [17]:
for lr in alpha_list: 
    alpha = lr # Learning rate 
    for g in gamma_list:
        qtable = np.zeros((state_size, action_size))
        total_episodes = 5000
        total_test_episodes = 100
        max_steps = 99 
        gamma = g # Discounting rate 
        epsilon = 1.0 # Exploration rate
        min_epsilon = 0.01
        max_epsilon = 1.0
        decay_rate = 0.01 # Exponential decay rate
        
        
        for episode in range(total_episodes):
            # Reset the environment
            state = env.reset()
            step = 0
            done = False
            for step in range(max_steps):
                exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
                if exp_exp_tradeoff > epsilon:
                    action = np.argmax(qtable[state,:])
        
        # Else doing a random choice --> exploration
                else:
                    action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
                new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
                qtable[state, action] = qtable[state, action] + alpha * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])
                
        # Our new state is state
                state = new_state
        
        # If done : finish episode
                if done == True: 
                    break
        env.reset()               # resets the environment and returns a random initial state

        rewards = []              

        for episode in range(total_test_episodes):
            state = env.reset()
            step = 0
            done = False
            total_rewards = 0
            #print("****************************************************")
            #print("EPISODE ", episode)

            for step in range(max_steps):
                # UNCOMMENT IT IF YOU WANT TO SEE OUR AGENT PLAYING
                #env.render()
                #Take the action (index) that have the maximum expected future reward given that state
                action = np.argmax(qtable[state,:])

                new_state, reward, done, info = env.step(action)

                total_rewards += reward

                if done:
                    rewards.append(total_rewards)
                    #print ("Score", total_rewards)
                    break
                state = new_state
        env.close()
        print("Alpha :",alpha)
        print("gamma :",g)
    
        print ("Score over time: " +  str(sum(rewards)/total_test_episodes))
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

Alpha : 0.6
gamma : 0.5
Score over time: 8.09
Alpha : 0.6
gamma : 0.54
Score over time: 8.02
Alpha : 0.6
gamma : 0.618
Score over time: 8.09
Alpha : 0.6
gamma : 0.71
Score over time: 7.79
Alpha : 0.6
gamma : 0.78
Score over time: 8.24
Alpha : 0.6
gamma : 0.88
Score over time: 8.16
Alpha : 0.65
gamma : 0.5
Score over time: 7.99
Alpha : 0.65
gamma : 0.54
Score over time: 7.75
Alpha : 0.65
gamma : 0.618
Score over time: 7.71
Alpha : 0.65
gamma : 0.71
Score over time: 7.81
Alpha : 0.65
gamma : 0.78
Score over time: 7.58
Alpha : 0.65
gamma : 0.88
Score over time: 7.53
Alpha : 0.7
gamma : 0.5
Score over time: 7.76
Alpha : 0.7
gamma : 0.54
Score over time: 7.68
Alpha : 0.7
gamma : 0.618
Score over time: 7.73
Alpha : 0.7
gamma : 0.71
Score over time: 7.97
Alpha : 0.7
gamma : 0.78
Score over time: 7.66
Alpha : 0.7
gamma : 0.88
Score over time: 8.08
Alpha : 0.75
gamma : 0.5
Score over time: 7.9
Alpha : 0.75
gamma : 0.54
Score over time: 7.61
Alpha : 0.75
gamma : 0.618
Score over time: 8.22
Alpha

#### Thus, the optimum value of alpha and gamma is 0.9 and 0.88 respectively for which the score over time is 8.31

In [22]:
epsilon_list = [0.01,0.1,0.2,0.8,1.0]
decay_rate_list = [0.01,0.1,0.2,0.6,1]

In [23]:
for ep in epsilon_list: 
    epsilon = ep # Exploration rate 
    for dr in decay_rate_list:
        qtable = np.zeros((state_size, action_size))
        total_episodes = 5000
        total_test_episodes = 100
        max_steps = 99
        alpha = 0.9  # Learning rate
        gamma = 0.88 # Discounting rate 
        min_epsilon = 0.01
        max_epsilon = 1.0
        decay_rate = dr # Exponential decay rate
        #print(decay_rate)
        
        
        for episode in range(total_episodes):
            # Reset the environment
            state = env.reset()
            step = 0
            done = False
            for step in range(max_steps):
                exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
                if exp_exp_tradeoff > epsilon:
                    action = np.argmax(qtable[state,:])
        
        # Else doing a random choice --> exploration
                else:
                    action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
                new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
                qtable[state, action] = qtable[state, action] + alpha * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])
                
        # Our new state is state
                state = new_state
        
        # If done : finish episode
                if done == True: 
                    break
        env.reset()               # resets the environment and returns a random initial state

        rewards = []              

        for episode in range(total_test_episodes):
            state = env.reset()
            step = 0
            done = False
            total_rewards = 0
            #print("****************************************************")
            #print("EPISODE ", episode)

            for step in range(max_steps):
                # UNCOMMENT IT IF YOU WANT TO SEE OUR AGENT PLAYING
                #env.render()
                #Take the action (index) that have the maximum expected future reward given that state
                action = np.argmax(qtable[state,:])

                new_state, reward, done, info = env.step(action)

                total_rewards += reward

                if done:
                    rewards.append(total_rewards)
                    #print ("Score", total_rewards)
                    break
                state = new_state
        env.close()
        print("Exploration Rate :",epsilon)
        print("Decay Rate :",dr)
    
        print ("Score over time: " +  str(sum(rewards)/total_test_episodes))
        print ("----------------------------------------------------------")
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

Exploration Rate : 0.01
Decay Rate : 0.01
Score over time: 7.67
----------------------------------------------------------
Exploration Rate : 0.01
Decay Rate : 0.1
Score over time: 7.82
----------------------------------------------------------
Exploration Rate : 0.01
Decay Rate : 0.2
Score over time: 7.54
----------------------------------------------------------
Exploration Rate : 0.01
Decay Rate : 0.6
Score over time: 8.32
----------------------------------------------------------
Exploration Rate : 0.01
Decay Rate : 1
Score over time: 7.53
----------------------------------------------------------
Exploration Rate : 0.1
Decay Rate : 0.01
Score over time: 7.45
----------------------------------------------------------
Exploration Rate : 0.1
Decay Rate : 0.1
Score over time: 7.83
----------------------------------------------------------
Exploration Rate : 0.1
Decay Rate : 0.2
Score over time: 8.13
----------------------------------------------------------
Exploration Rate : 0.1
Deca

#### Thus, the optimum value of Exploration Rate  and Decay Rate is 0.8 and 0.2 respectively for which the score over time is 8.51

### So far, the % increase from the baseline performance is ((7.32-8.51)/7.32)*100 that equals : 16.25%   😲


Let us now tune the number of episodes

In [40]:
total_episodes_list = [5000,10000,25000,28000,50000,70000]

In [43]:
    for tel in total_episodes_list:
        total_episodes = tel
        #print(dr)
        qtable = np.zeros((state_size, action_size))
        
        #total_episodes = 5000
        total_test_episodes = 100
        max_steps = 99
        
        alpha = 0.9
        gamma = 0.88 # Discounting rate 
        
        epsilon = 0.8 # Exploration rate
        min_epsilon = 0.01
        max_epsilon = 1.0
        decay_rate = 0.2 # Exponential decay rate
        #print(decay_rate)
        
        
        for episode in range(total_episodes):
            # Reset the environment
            state = env.reset()
            step = 0
            done = False
            for step in range(max_steps):
                exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
                if exp_exp_tradeoff > epsilon:
                    action = np.argmax(qtable[state,:])
        
        # Else doing a random choice --> exploration
                else:
                    action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
                new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
                qtable[state, action] = qtable[state, action] + alpha * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])
                
        # Our new state is state
                state = new_state
        
        # If done : finish episode
                if done == True: 
                    break
        env.reset()               # resets the environment and returns a random initial state

        rewards = []              

        for episode in range(total_test_episodes):
            state = env.reset()
            step = 0
            done = False
            total_rewards = 0
            #print("****************************************************")
            #print("EPISODE ", episode)

            for step in range(max_steps):
                # UNCOMMENT IT IF YOU WANT TO SEE OUR AGENT PLAYING
                #env.render()
                #Take the action (index) that have the maximum expected future reward given that state
                action = np.argmax(qtable[state,:])

                new_state, reward, done, info = env.step(action)

                total_rewards += reward

                if done:
                    rewards.append(total_rewards)
                    #print ("Score", total_rewards)
                    break
                state = new_state
        env.close()
        print("Total Episodes :",total_episodes)
        #print("Decay Rate :",dr)
    
        print ("Score over time: " +  str(sum(rewards)/total_test_episodes))
        print ("----------------------------------------------------------")
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

Total Episodes : 5000
Score over time: 7.86
----------------------------------------------------------
Total Episodes : 10000
Score over time: 7.94
----------------------------------------------------------
Total Episodes : 25000
Score over time: 7.85
----------------------------------------------------------
Total Episodes : 28000
Score over time: 7.68
----------------------------------------------------------
Total Episodes : 50000
Score over time: 7.46
----------------------------------------------------------
Total Episodes : 70000
Score over time: 8.25
----------------------------------------------------------


### The score over time for 70000 number of episodes is 8.25

## Resulting in the % increase from the baseline of 12.70%

# --------------------------------------------------------------------------------------------------------------

### Attempt to try for policy other than argmax

In [62]:
total_episodes = 5000
total_test_episodes = 100
max_steps = 99 
alpha= 0.9 # Learning rate 
gamma = 0.88 # Discounting rate 
epsilon = 0.8 # Exploration rate
min_epsilon = 0.01
max_epsilon = 1.0
decay_rate = 0.2 # Exponential decay rate

In [63]:
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
        
        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + alpha * (reward + gamma * 
                                    np.min(qtable[new_state, :]) - qtable[state, action])
                
        # Our new state is state
        state = new_state
        
        # If done : finish episode
        if done == True: 
            break
    
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

In [65]:
env.reset()               # resets the environment and returns a random initial state
rewards = []              

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    #print("****************************************************")
    #print("EPISODE ", episode)

    for step in range(max_steps):
        # UNCOMMENT IT IF YOU WANT TO SEE OUR AGENT PLAYING
        # env.render()
        #Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Score over time: 4.37


# --------------------------------------------------------------------------------------------------------------

# --------------------------------------------------------------------------------------------------------------

# Conclusion

From this notebook, we can conclude, that the score over time of an algorithm, in an environment can be imrpoved by tuning the hyperparameters. Trying out through the various values of hyperparameter and iterating them is one of the ways. 

# --------------------------------------------------------------------------------------------------------------

# Author :

Kompi Sadasivappa, Indupriya

MS Information Systems, 

Northeastern University, Boston


# Citation

https://github.com/simoninithomas/Deep_reinforcement_learning_Course

https://www.youtube.com/results?search_query=Q-learning+with+numpy+and+OpenAI+Taxi-v2

https://www.youtube.com/playlist?list=PLQLZ37V8CnUTdIoJJdvmmFoQJntZ9dp5Q

https://github.com/Lodur03/Q-Learning-Taxi-v2/blob/master/Q-Learning-Taxi.ipynb

https://www.biostat.wisc.edu/~craven/cs760/lectures/reinforcement.pdf

https://towardsdatascience.com/

# Licensing

Copyright 2020 Indupriya Kompi Sadasivappa

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.